In [1]:
from urllib import robotparser
import pandas as pd
import time
import numpy as np
import random

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ChromeOptions, ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
class NBAScraper():

    def __init__(self, user_agent: str):
        self.rp = robotparser.RobotFileParser()
        self.user_agent = user_agent
        self.base_url = "https://www.nba.com/"
        
        self.output = None

        # TODO: Implementar proxy para que la pagina no me bloquee cuando hago muchas ejecuciones

        # https://developer.chrome.com/docs/chromedriver/capabilities?hl=es-419
        webdriver_options = ChromeOptions()
        # webdriver_options.add_argument("--headless")
        webdriver_options.add_argument(f"--user-agent={user_agent}")
        # evitar deteccion como bot (https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958)
        webdriver_options.add_argument('--disable-blink-features=AutomationControlled')
        self.driver = webdriver.Chrome(options=webdriver_options)

        # Configuración de timeouts del navegador
        self.driver.set_page_load_timeout(60)  # Máximo 20 segundos para cargar una página
        self.driver.implicitly_wait(5)  # Espera máxima para encontrar elementos antes de lanzar error
        

    # Comprobar accesibilidad al sitio web
    def check_accessibility(self, url_robots: str, url: str) -> bool:
        # https://docs.python.org/3/library/urllib.robotparser.html

        # Establece en el parser la url a robots.txt
        self.rp.set_url(url=url_robots)
        # Parsea robots.txt
        self.rp.read()
        # Examina que user_agent pueda acceder en base al robots.txt parseado
        return self.rp.can_fetch(useragent=self.user_agent, url=url)
    

    
    def extract_teams_shooting_ds(self) -> None:
        url_robots = f"{self.base_url}robots.txt"
        url_stats = f"{self.base_url}stats/teams/shooting"

        def get_table_contents(html, season, conference, position):
            # Creamos objeto BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Extraemos la tabla de estadisticas
            teams_table = soup.find("table", class_="Crom_table__p1iZz")

            # Obtenemos las columnas de cada estadistica
            table_head = teams_table.find("thead")

            # Crom_headers__mzI_m -> Cabecera perteneciente a FGM, FGA, FG%
            # el apartado field contiene adicionalmente a que rango de tiro pertenece
            categories = [col.get("field") for col in table_head.find("tr", class_="Crom_headers__mzI_m").find_all("th")[1:]]
            df = pd.DataFrame(columns=["Team", "Season", "Conference", "Position"] + categories)
            table_body = teams_table.find("tbody").find_all("tr")
            for tr in table_body:
                team_info = tr.find_all("td")
                team_name = team_info[0].get_text().strip()
                team_stats = [float(stats.get_text()) for stats in team_info[1:]]
                
                # Categorias y estadisticas deben tener la misma longitud, deberia cumplirse siempre
                assert len(categories) == len(team_stats)
                df = pd.concat([df, pd.DataFrame(data=np.array([[team_name, season, conference, position] + team_stats]), columns=df.columns)])
            
            return df
        

        final_df = None


        # Comprobamos accesibilidad a la pagina
        if self.check_accessibility(url_robots=url_robots, url=url_stats):
            print(f"{url_stats} visited!")
            self.driver.get(url_stats)
            wait = WebDriverWait(self.driver, 20)
           
            try:
                # https://stackoverflow.com/questions/64032271/handling-accept-cookies-popup-with-selenium-in-python
                print("Waiting...")
                wait.until(
                    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
                ).click()
                print("Cookies accepted!")
            except:
                print("Wait failed!")
                self.quit_driver()
                return
            
            # Esperamos para que la accion de aceptar las cookies no se solape con empezar la extraccion de las tablas, o el programa peta
            time.sleep(5)

            # Cargamos los filtros avanzados para poder hacer búsqueda por conferencias
            try:
                print("Intentando abrir Advanced Filters...")
                toggle_button = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button.StatsAdvancedFiltersPanel_safArrow__EqRgu")
                ))
                toggle_button.click()
                time.sleep(3)  # Pequeña espera para que los filtros se desplieguen correctamente
                print("Advanced Filters abiertos.")
            except Exception as e:
                print("No se pudo abrir el panel de filtros avanzados.")
                print(str(e))
                self.quit_driver()
                return

            # Obtenemos el boton para aplicar los filtros
            get_stats_button = None
            try:
                for button in self.driver.find_elements(by=By.CLASS_NAME, value="Button_button__L2wUb"):
                    if button.text.lower() == "get stats":
                        get_stats_button = button
            except:
                print("No se pudo acceder a los botones.")
                self.quit_driver()
                return
            
            # Nos aseguramos de tener boton para aplicar los filtros
            assert get_stats_button != None
            
            # Obtenemos el elemento que contiene todos los filtros
            overall_filters = self.driver.find_element(by=By.CLASS_NAME, value="nba-stats-primary-split-block")
            # Obtenemos los filtros que aparecen al inicio
            initial_filters = overall_filters.find_elements(by=By.CLASS_NAME, value="DropDown_label__lttfI")

            # Obtenemos todas las temporadas
            season_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "SEASON":
                    season_options = filt.find_elements(by=By.TAG_NAME, value="option")
                    break
            
            # Obtenemos todas las conferencias
            conference_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "CONFERENCE":
                        conference_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["East", "West"]
                        ]
                        break
            
            # Obtenemos todas las posiciones de jugador
            positions_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "POSITION":
                        positions_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["Center", "Guard", "Forward"]
                        ]
                        break
   

            # Iteramos por todas las combinaciones de filtros
            for season_op in season_options:
                season_text = season_op.text
                try:
                    wait.until(EC.element_to_be_clickable(season_op)).click()
                    time.sleep(1)
                except:
                    print("Wait failed!")
                    self.quit_driver()
                
                for conf_op in conference_options:
                    conf_text = conf_op.text
                    try:
                        wait.until(EC.element_to_be_clickable(conf_op)).click()
                        time.sleep(1)
                    except:
                        print("Wait failed!")
                        self.quit_driver()
                    
                    for pos_op in positions_options:
                        pos_text = pos_op.text
                        time.sleep(1)
                        try:
                            wait.until(EC.element_to_be_clickable(pos_op)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()
                        
                        # Aplicamos los filtros
                        try:
                            wait.until(EC.element_to_be_clickable(get_stats_button)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()

                        time.sleep(1)
                        # Esperamos a que la tabla sea visible
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "Crom_table__p1iZz")))
                        # Extraemos el dataframe inicial
                        html = self.driver.page_source

                        curr_df = get_table_contents(html=html, season=season_text, conference=conf_text, position=pos_text)
                        if final_df is None:
                            final_df = curr_df
                        else:
                            final_df = pd.concat([final_df, curr_df])

                        # Añadimos espaciado de peticiones HTTP:
                        # Introducimos un retraso un retraso aleatorio, así simulamos comportamiento más humano
                        response_delay = random.uniform(1.0, 1.5)
                        print(f"[INFO] Esperando {response_delay:.2f} segundos antes de la siguiente petición...")
                        time.sleep(response_delay)
        
        return final_df

    

    def extract_teams_contested_shoots(self, check_accept_cookies=False):
        url_robots = f"{self.base_url}robots.txt"
        url_stats = f"{self.base_url}stats/teams/hustle"

        def get_table_contents(html, season, conference, position):
            # Creamos objeto BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Extraemos la tabla de estadisticas
            teams_table = soup.find("table", class_="Crom_table__p1iZz")

            # Obtenemos las columnas de cada estadistica
            table_head = teams_table.find("thead")

            # Categorias que nos interesan
            cats_of_interest = ["contested_shots_2pt", "contested_shots_3pt"]

            # Crom_headers__mzI_m -> Cabecera de la tabla
            categories = [col.get("field").lower() for col in table_head.find("tr", class_="Crom_headers__mzI_m").find_all("th")[1:]]

            # Indices de las categorias de interes dentro de todas las categoria 
            categories_idx = [categories.index(c) for c in cats_of_interest]

            df = pd.DataFrame(columns=["Team", "Season", "Conference", "Position"] + cats_of_interest)
            table_body = teams_table.find("tbody").find_all("tr")
            for tr in table_body:
                team_info = tr.find_all("td")
                team_name = team_info[0].get_text().strip()
                # Cogemos unicamente las estadisticas de interes
                team_stats = [float(team_info[1:][i].get_text()) for i in categories_idx]
                
                # Categorias y estadisticas deben tener la misma longitud, deberia cumplirse siempre
                assert len(categories_idx) == len(team_stats)
                print(team_stats)
                df = pd.concat([df, pd.DataFrame(data=np.array([[team_name, season, conference, position] + team_stats]), columns=df.columns)])
            
            return df


        final_df = None
        
        # Comprobamos accesibilidad a la pagina
        if self.check_accessibility(url_robots=url_robots, url=url_stats):
            print(f"{url_stats} visited!")
            self.driver.get(url_stats)
            wait = WebDriverWait(self.driver, 20)

            # No aparecerá el boton de aceptar cookies si ya se ha aceptado en el link anterior
            if check_accept_cookies:
                try:
                    # https://stackoverflow.com/questions/64032271/handling-accept-cookies-popup-with-selenium-in-python
                    print("Waiting...")
                    wait.until(
                        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
                    ).click()
                    print("Cookies accepted!")
                except:
                    print("Wait failed!")
                    self.quit_driver()
                    return
            
            # Esperamos para que la accion de aceptar las cookies no se solape con empezar la extraccion de las tablas, o el programa peta
            time.sleep(5)

            # Cargamos los filtros avanzados para poder hacer búsqueda por conferencias
            try:
                print("Intentando abrir Advanced Filters...")
                toggle_button = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button.StatsAdvancedFiltersPanel_safArrow__EqRgu")
                ))
                toggle_button.click()
                time.sleep(3)  # Pequeña espera para que los filtros se desplieguen correctamente
                print("Advanced Filters abiertos.")
            except Exception as e:
                print("No se pudo abrir el panel de filtros avanzados.")
                print(str(e))
                self.quit_driver()
                return

            # Obtenemos el boton para aplicar los filtros
            get_stats_button = None
            try:
                for button in self.driver.find_elements(by=By.CLASS_NAME, value="Button_button__L2wUb"):
                    if button.text.lower() == "get stats":
                        get_stats_button = button
            except:
                print("No se pudo acceder a los botones.")
                self.quit_driver()
                return
            
            # Nos aseguramos de tener boton para aplicar los filtros
            assert get_stats_button != None
            
            # Obtenemos el elemento que contiene todos los filtros
            overall_filters = self.driver.find_element(by=By.CLASS_NAME, value="nba-stats-primary-split-block")
            # Obtenemos los filtros que aparecen al inicio
            initial_filters = overall_filters.find_elements(by=By.CLASS_NAME, value="DropDown_label__lttfI")

            # Obtenemos todas las temporadas
            season_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "SEASON":
                    season_options = filt.find_elements(by=By.TAG_NAME, value="option")
                    break
            
            # Obtenemos todas las conferencias
            conference_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "CONFERENCE":
                        conference_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["East", "West"]
                        ]
                        break
            
            # Obtenemos todas las posiciones de jugador
            positions_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "POSITION":
                        positions_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["Center", "Guard", "Forward"]
                        ]
                        break
            

            # Iteramos por todas las combinaciones de filtros
            for season_op in season_options:
                season_text = season_op.text
                try:
                    wait.until(EC.element_to_be_clickable(season_op)).click()
                    time.sleep(1)
                except:
                    print("Wait failed!")
                    self.quit_driver()
                
                for conf_op in conference_options:
                    conf_text = conf_op.text
                    try:
                        wait.until(EC.element_to_be_clickable(conf_op)).click()
                        time.sleep(1)
                    except:
                        print("Wait failed!")
                        self.quit_driver()
                    
                    for pos_op in positions_options:
                        pos_text = pos_op.text
                        time.sleep(1)
                        try:
                            wait.until(EC.element_to_be_clickable(pos_op)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()
                        
                        # Aplicamos los filtros
                        try:
                            wait.until(EC.element_to_be_clickable(get_stats_button)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()

                        time.sleep(1)
                        # Esperamos a que la tabla sea visible
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "Crom_table__p1iZz")))
                        # Extraemos el dataframe inicial
                        html = self.driver.page_source

                        curr_df = get_table_contents(html=html, season=season_text, conference=conf_text, position=pos_text)
                        if final_df is None:
                            final_df = curr_df
                        else:
                            final_df = pd.concat([final_df, curr_df])

                        # Añadimos espaciado de peticiones HTTP:
                        # Introducimos un retraso un retraso aleatorio, así simulamos comportamiento más humano
                        response_delay = random.uniform(1.0, 1.5)
                        print(f"[INFO] Esperando {response_delay:.2f} segundos antes de la siguiente petición...")
                        time.sleep(response_delay)
            

        return final_df   


    def extract_teams_boxouts(self, check_accept_cookies=False) -> pd.DataFrame:
        url_robots = f"{self.base_url}robots.txt"
        url_stats = f"{self.base_url}stats/teams/box-outs"

        def get_table_contents(html, season, conference, position):
            # Creamos objeto BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Extraemos la tabla de estadisticas
            teams_table = soup.find("table", class_="Crom_table__p1iZz")

            # Obtenemos las columnas de cada estadistica
            table_head = teams_table.find("thead")

            # Categorias que nos interesan
            cats_of_interest = ["off_boxouts", "def_boxouts"]

            # Crom_headers__mzI_m -> Cabecera de la tabla
            categories = [col.get("field").lower() for col in table_head.find("tr", class_="Crom_headers__mzI_m").find_all("th")[1:]]

            # Indices de las categorias de interes dentro de todas las categoria 
            categories_idx = [categories.index(c) for c in cats_of_interest]

            df = pd.DataFrame(columns=["Team", "Season", "Conference", "Position"] + cats_of_interest)
            table_body = teams_table.find("tbody").find_all("tr")
            for tr in table_body:
                team_info = tr.find_all("td")
                team_name = team_info[0].get_text().strip()
                # Cogemos unicamente las estadisticas de interes
                team_stats = [float(team_info[1:][i].get_text()) for i in categories_idx]
                
                # Categorias y estadisticas deben tener la misma longitud, deberia cumplirse siempre
                assert len(categories_idx) == len(team_stats)
                print(team_stats)
                df = pd.concat([df, pd.DataFrame(data=np.array([[team_name, season, conference, position] + team_stats]), columns=df.columns)])
            
            return df


        final_df = None
        
        # Comprobamos accesibilidad a la pagina
        if self.check_accessibility(url_robots=url_robots, url=url_stats):
            print(f"{url_stats} visited!")
            self.driver.get(url_stats)
            wait = WebDriverWait(self.driver, 20)

            # No aparecerá el boton de aceptar cookies si ya se ha aceptado en el link anterior
            if check_accept_cookies:
                try:
                    # https://stackoverflow.com/questions/64032271/handling-accept-cookies-popup-with-selenium-in-python
                    print("Waiting...")
                    wait.until(
                        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
                    ).click()
                    print("Cookies accepted!")
                except:
                    print("Wait failed!")
                    self.quit_driver()
                    return
            
            # Esperamos para que la accion de aceptar las cookies no se solape con empezar la extraccion de las tablas, o el programa peta
            time.sleep(5)

            # Cargamos los filtros avanzados para poder hacer búsqueda por conferencias
            try:
                print("Intentando abrir Advanced Filters...")
                toggle_button = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button.StatsAdvancedFiltersPanel_safArrow__EqRgu")
                ))
                toggle_button.click()
                time.sleep(3)  # Pequeña espera para que los filtros se desplieguen correctamente
                print("Advanced Filters abiertos.")
            except Exception as e:
                print("No se pudo abrir el panel de filtros avanzados.")
                print(str(e))
                self.quit_driver()
                return

            # Obtenemos el boton para aplicar los filtros
            get_stats_button = None
            try:
                for button in self.driver.find_elements(by=By.CLASS_NAME, value="Button_button__L2wUb"):
                    if button.text.lower() == "get stats":
                        get_stats_button = button
            except:
                print("No se pudo acceder a los botones.")
                self.quit_driver()
                return
            
            # Nos aseguramos de tener boton para aplicar los filtros
            assert get_stats_button != None
            
            # Obtenemos el elemento que contiene todos los filtros
            overall_filters = self.driver.find_element(by=By.CLASS_NAME, value="nba-stats-primary-split-block")
            # Obtenemos los filtros que aparecen al inicio
            initial_filters = overall_filters.find_elements(by=By.CLASS_NAME, value="DropDown_label__lttfI")

            # Obtenemos todas las temporadas
            season_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "SEASON":
                    season_options = filt.find_elements(by=By.TAG_NAME, value="option")
                    break
            
            # Obtenemos todas las conferencias
            conference_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "CONFERENCE":
                        conference_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["East", "West"]
                        ]
                        break
            
            # Obtenemos todas las posiciones de jugador
            positions_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "POSITION":
                        positions_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["Center", "Guard", "Forward"]
                        ]
                        break
            

            # Iteramos por todas las combinaciones de filtros
            for season_op in season_options:
                season_text = season_op.text
                try:
                    wait.until(EC.element_to_be_clickable(season_op)).click()
                    time.sleep(1)
                except:
                    print("Wait failed!")
                    self.quit_driver()
                
                for conf_op in conference_options:
                    conf_text = conf_op.text
                    try:
                        wait.until(EC.element_to_be_clickable(conf_op)).click()
                        time.sleep(1)
                    except:
                        print("Wait failed!")
                        self.quit_driver()
                    
                    for pos_op in positions_options:
                        pos_text = pos_op.text
                        time.sleep(1)
                        try:
                            wait.until(EC.element_to_be_clickable(pos_op)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()
                        
                        # Aplicamos los filtros
                        try:
                            wait.until(EC.element_to_be_clickable(get_stats_button)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()

                        time.sleep(1)
                        # Esperamos a que la tabla sea visible
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "Crom_table__p1iZz")))
                        # Extraemos el dataframe inicial
                        html = self.driver.page_source

                        curr_df = get_table_contents(html=html, season=season_text, conference=conf_text, position=pos_text)
                        if final_df is None:
                            final_df = curr_df
                        else:
                            final_df = pd.concat([final_df, curr_df])

                        # Añadimos espaciado de peticiones HTTP:
                        # Introducimos un retraso un retraso aleatorio, así simulamos comportamiento más humano
                        response_delay = random.uniform(1.0, 1.5)
                        print(f"[INFO] Esperando {response_delay:.2f} segundos antes de la siguiente petición...")
                        time.sleep(response_delay)
            

        return final_df   


    def extract_data(self, sub_url: str, cats_of_interest: list[str] = [], check_accept_cookies=True) -> pd.DataFrame:
        url_robots = f"{self.base_url}robots.txt"
        url_stats = f"{self.base_url}stats{sub_url}"

        def get_table_contents(html, season: str, conference: str, position: str, cats_of_interest: list[str] = []):
            # Creamos objeto BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Extraemos la tabla de estadisticas
            teams_table = soup.find("table", class_="Crom_table__p1iZz")

            # Obtenemos las columnas de cada estadistica
            table_head = teams_table.find("thead")

            # Crom_headers__mzI_m -> Cabecera de la tabla
            categories = [col.get("field").lower() for col in table_head.find("tr", class_="Crom_headers__mzI_m").find_all("th")[1:]]

            categories_idx = range(len(categories))
            if len(cats_of_interest) > 0:
                # Indices de las categorias de interes dentro de todas las categoria 
                categories_idx = [categories.index(c) for c in cats_of_interest]
            
            if len(cats_of_interest) == 0:
                cats_of_interest = [categories[i] for i in categories_idx]

            df = pd.DataFrame(columns=["Team", "Season", "Conference", "Position"] + cats_of_interest)
            table_body = teams_table.find("tbody").find_all("tr")
            for tr in table_body:
                team_info = tr.find_all("td")
                team_name = team_info[0].get_text().strip()
                # Cogemos unicamente las estadisticas de interes
                team_stats = [float(team_info[1:][i].get_text()) for i in categories_idx]
                
                # Categorias y estadisticas deben tener la misma longitud, deberia cumplirse siempre
                assert len(categories_idx) == len(team_stats)
                print(team_stats)
                df = pd.concat([df, pd.DataFrame(data=np.array([[team_name, season, conference, position] + team_stats]), columns=df.columns)])
            
            return df

        
        final_df = None

        # Comprobamos accesibilidad a la pagina
        if self.check_accessibility(url_robots=url_robots, url=url_stats):
            print(f"{url_stats} visited!")
            self.driver.get(url_stats)
            wait = WebDriverWait(self.driver, 20)

            # No aparecerá el boton de aceptar cookies si ya se ha aceptado en el link anterior
            if check_accept_cookies:
                try:
                    # https://stackoverflow.com/questions/64032271/handling-accept-cookies-popup-with-selenium-in-python
                    print("Waiting...")
                    wait.until(
                        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
                    ).click()
                    print("Cookies accepted!")
                except:
                    print("Wait failed!")
                    self.quit_driver()
                    return
            
            # Esperamos para que la accion de aceptar las cookies no se solape con empezar la extraccion de las tablas, o el programa peta
            time.sleep(5)

            # Cargamos los filtros avanzados para poder hacer búsqueda por conferencias
            try:
                print("Intentando abrir Advanced Filters...")
                toggle_button = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "button.StatsAdvancedFiltersPanel_safArrow__EqRgu")
                ))
                toggle_button.click()
                time.sleep(3)  # Pequeña espera para que los filtros se desplieguen correctamente
                print("Advanced Filters abiertos.")
            except Exception as e:
                print("No se pudo abrir el panel de filtros avanzados.")
                print(str(e))
                self.quit_driver()
                return

            # Obtenemos el boton para aplicar los filtros
            get_stats_button = None
            try:
                for button in self.driver.find_elements(by=By.CLASS_NAME, value="Button_button__L2wUb"):
                    if button.text.lower() == "get stats":
                        get_stats_button = button
            except:
                print("No se pudo acceder a los botones.")
                self.quit_driver()
                return
            
            # Nos aseguramos de tener boton para aplicar los filtros
            assert get_stats_button != None
            
            # Obtenemos el elemento que contiene todos los filtros
            overall_filters = self.driver.find_element(by=By.CLASS_NAME, value="nba-stats-primary-split-block")
            # Obtenemos los filtros que aparecen al inicio
            initial_filters = overall_filters.find_elements(by=By.CLASS_NAME, value="DropDown_label__lttfI")

            # Obtenemos todas las temporadas
            season_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "SEASON":
                    season_options = filt.find_elements(by=By.TAG_NAME, value="option")
                    break
            
            # Obtenemos todas las conferencias
            conference_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "CONFERENCE":
                        conference_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["East", "West"]
                        ]
                        break
            
            # Obtenemos todas las posiciones de jugador
            positions_options = []
            for filt in initial_filters:
                tag = filt.find_element(by=By.TAG_NAME, value="p").text
                if tag.upper() == "POSITION":
                        positions_options = [
                            opt for opt in filt.find_elements(by=By.TAG_NAME, value="option")
                            if opt.text in ["Center", "Guard", "Forward"]
                        ]
                        break
            

            # Iteramos por todas las combinaciones de filtros
            for season_op in season_options:
                season_text = season_op.text
                try:
                    wait.until(EC.element_to_be_clickable(season_op)).click()
                    time.sleep(1)
                except:
                    print("Wait failed!")
                    self.quit_driver()
                
                for conf_op in conference_options:
                    conf_text = conf_op.text
                    try:
                        wait.until(EC.element_to_be_clickable(conf_op)).click()
                        time.sleep(1)
                    except:
                        print("Wait failed!")
                        self.quit_driver()
                    
                    for pos_op in positions_options:
                        pos_text = pos_op.text
                        time.sleep(1)
                        try:
                            wait.until(EC.element_to_be_clickable(pos_op)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()
                        
                        # Aplicamos los filtros
                        try:
                            wait.until(EC.element_to_be_clickable(get_stats_button)).click()
                        except:
                            print("Wait failed!")
                            self.quit_driver()

                        time.sleep(1)
                        # Esperamos a que la tabla sea visible
                        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "Crom_table__p1iZz")))
                        # Extraemos el dataframe inicial
                        html = self.driver.page_source

                        curr_df = get_table_contents(html=html, season=season_text, conference=conf_text, position=pos_text, cats_of_interest=cats_of_interest)
                        if final_df is None:
                            final_df = curr_df
                        else:
                            final_df = pd.concat([final_df, curr_df])

                        # Añadimos espaciado de peticiones HTTP:
                        # Introducimos un retraso un retraso aleatorio, así simulamos comportamiento más humano
                        response_delay = random.uniform(1.0, 1.5)
                        print(f"[INFO] Esperando {response_delay:.2f} segundos antes de la siguiente petición...")
                        time.sleep(response_delay)
            

        return final_df




    def execute_scraping(self):
        df1 = self.extract_teams_shooting_ds()
        df2 = self.extract_teams_contested_shoots(check_accept_cookies=False)
        df1 = df1.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)
        df3 = pd.merge(df1, df2, on=["Team", "Season", "Conference", "Position"], how="left")

        df4 = self.extract_teams_boxouts(check_accept_cookies=False)
        df4 = df4.reset_index(drop=True)
        
        for df in [df3, df4]:
            for col in ["Team", "Season", "Conference", "Position"]:
                df[col] = df[col].astype(str)
        
        print(df3)
        print(df4)
        self.output = pd.merge(df3, df4, on=["Team", "Season", "Conference", "Position"], how="left")     
        print(self.output) 
    

    def execute_scraping_2(self):
        df1 = self.extract_data(sub_url="/teams/shooting", check_accept_cookies=True)
        df2 = self.extract_data(sub_url="/teams/hustle", cats_of_interest=["contested_shots_2pt", "contested_shots_3pt"], check_accept_cookies=False)
        df1 = df1.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)

        df3 = pd.merge(df1, df2, on=["Team", "Season", "Conference", "Position"], how="left")

        df4 = self.extract_data(sub_url="/teams/box-outs", cats_of_interest=["off_boxouts", "def_boxouts"], check_accept_cookies=False)
        df4 = df4.reset_index(drop=True)

        print(df3)
        print(df4)
        self.output = pd.merge(df3, df4, on=["Team", "Season", "Conference", "Position"], how="left")
        print(self.output) 


    # Cerrar driver
    def quit_driver(self):
        self.driver.quit()


    # Guardar resultados en ambos formatos de csv
    def get_csv(self):
        self.output.to_csv("..\\..\\dataset\\nba_test_dataset.csv", sep=",", index=False)
        self.output.to_csv("..\\..\\dataset\\nba_test_dataset_excel.csv", sep=";", index=False)
        

In [11]:
user_agent_windows = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
scraper = NBAScraper(user_agent=user_agent_windows)

df1 = scraper.extract_data(sub_url="/teams/shooting", check_accept_cookies=True)
df2 = scraper.extract_data(sub_url="/teams/hustle", cats_of_interest=["contested_shots_2pt", "contested_shots_3pt"], check_accept_cookies=False)
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

df3 = pd.merge(df1, df2, on=["Team", "Season", "Conference", "Position"], how="left")

df4 = scraper.extract_data(sub_url="/teams/box-outs", cats_of_interest=["off_boxouts", "def_boxouts"], check_accept_cookies=False)
df4 = df4.reset_index(drop=True)

output = pd.merge(df3, df4, on=["Team", "Season", "Conference", "Position"], how="left")

scraper.quit_driver()

https://www.nba.com/stats/teams/shooting visited!
Waiting...
Cookies accepted!
Intentando abrir Advanced Filters...
Advanced Filters abiertos.
[11.0, 16.7, 65.7, 2.6, 5.9, 44.5, 0.9, 2.4, 38.4, 0.4, 0.9, 50.0, 3.4, 9.1, 37.2, 2.9, 8.1, 35.5]
[10.9, 16.1, 67.4, 2.3, 5.1, 45.2, 1.7, 4.1, 41.3, 1.4, 3.4, 40.0, 2.9, 7.2, 39.7, 6.6, 18.7, 35.5]
[8.1, 13.3, 61.0, 1.9, 5.3, 36.6, 0.9, 2.4, 37.2, 0.4, 1.0, 36.4, 3.7, 10.5, 35.5, 5.0, 14.0, 35.7]
[7.6, 12.4, 61.2, 2.0, 4.9, 40.5, 0.6, 1.9, 30.7, 0.4, 1.0, 34.2, 2.0, 5.7, 35.5, 3.3, 10.4, 32.1]
[6.3, 10.4, 60.0, 1.3, 2.8, 47.0, 0.2, 0.7, 33.9, 0.1, 0.5, 28.9, 2.4, 6.6, 37.0, 2.9, 9.0, 32.6]
[4.1, 6.6, 61.3, 1.7, 3.1, 54.8, 1.0, 1.9, 51.4, 0.2, 0.4, 48.5, 3.1, 7.9, 39.0, 3.2, 8.2, 39.2]
[9.4, 14.5, 65.2, 2.2, 5.1, 42.9, 1.4, 2.9, 46.9, 0.5, 1.6, 31.1, 2.3, 7.1, 33.2, 2.8, 8.3, 33.3]
[14.2, 20.7, 68.8, 3.1, 6.8, 45.5, 2.7, 5.7, 47.1, 1.5, 3.6, 41.3, 3.7, 9.2, 40.0, 5.7, 15.6, 36.6]
[7.5, 11.8, 63.9, 2.7, 6.1, 43.6, 1.8, 4.4, 42.2, 0.7, 1.6, 46.3, 

In [12]:
scraper.quit_driver()

In [13]:
df1.describe()

,Team,Season,Conference,Position,less than 5 ft. fgm,less than 5 ft. fga,less than 5 ft. fg pct,5-9 ft. fgm,5-9 ft. fga,5-9 ft. fg pct,...,10-14 ft. fg pct,15-19 ft. fgm,15-19 ft. fga,15-19 ft. fg pct,20-24 ft. fgm,20-24 ft. fga,20-24 ft. fg pct,25-29 ft. fgm,25-29 ft. fga,25-29 ft. fg pct
count,2585,2585,2585,2585,2585,2585,2585,2585,2585,2585,...,2585,2585,2585,2585,2585,2585,2585,2585,2585,2585
unique,38,29,2,3,148,227,253,44,92,278,...,295,71,151,258,69,158,267,81,199,244
top,Atlanta Hawks,2024-25,East,Center,6.6,11.7,58.8,1.0,2.5,40.5,...,39.5,1.0,0.2,40.6,0.0,0.1,0.0,0.0,0.0,0.0
freq,88,90,1304,862,55,36,33,165,77,35,...,41,80,39,39,233,108,76,511,344,311


In [14]:
df2.describe()

,Team,Season,Conference,Position,contested_shots_2pt,contested_shots_3pt
count,854,854,854,854,854,854
unique,30,10,2,3,208,158
top,San Antonio Spurs,2024-25,West,Forward,13.4,11.3
freq,32,90,428,285,14,17


In [15]:
df4.describe()

,Team,Season,Conference,Position,off_boxouts,def_boxouts
count,719,719,719,719,719,719
unique,30,8,2,3,46,148
top,Atlanta Hawks,2024-25,East,Forward,0.1,2.7
freq,24,90,360,240,101,20


In [ ]:
df3 = pd.merge(df1, df2, on=["Team", "Season", "Conference", "Position"], how="left")

In [16]:
df3.describe()

,Team,Season,Conference,Position,less than 5 ft. fgm,less than 5 ft. fga,less than 5 ft. fg pct,5-9 ft. fgm,5-9 ft. fga,5-9 ft. fg pct,...,15-19 ft. fga,15-19 ft. fg pct,20-24 ft. fgm,20-24 ft. fga,20-24 ft. fg pct,25-29 ft. fgm,25-29 ft. fga,25-29 ft. fg pct,contested_shots_2pt,contested_shots_3pt
count,2585,2585,2585,2585,2585,2585,2585,2585,2585,2585,...,2585,2585,2585,2585,2585,2585,2585,2585,735,735
unique,38,29,2,3,148,227,253,44,92,278,...,151,258,69,158,267,81,199,244,203,156
top,Atlanta Hawks,2024-25,East,Center,6.6,11.7,58.8,1.0,2.5,40.5,...,0.2,40.6,0.0,0.1,0.0,0.0,0.0,0.0,8.0,11.3
freq,88,90,1304,862,55,36,33,165,77,35,...,39,39,233,108,76,511,344,311,12,14


In [17]:
df3.iloc[0]["Season"] == df4.iloc[0]["Season"] and \
df3.iloc[0]["Team"] == df4.iloc[0]["Team"] and \
df3.iloc[0]["Conference"] == df4.iloc[0]["Conference"] and \
df3.iloc[0]["Position"] == df4.iloc[0]["Position"]

True

In [18]:
print(df3.iloc[0]["Team"])
print(df4.iloc[0]["Team"])

Atlanta Hawks
Atlanta Hawks


In [ ]:
for df in [df3, df4]:
    for col in ["Team", "Season", "Conference", "Position"]:
        df[col] = df[col].astype(str)

output = pd.merge(df3, df4, on=["Team", "Season", "Conference", "Position"], how="left")
output.head(n=5)

,Team,Season,Conference,Position,less than 5 ft. fgm,less than 5 ft. fga,less than 5 ft. fg pct,5-9 ft. fgm,5-9 ft. fga,5-9 ft. fg pct,...,20-24 ft. fgm,20-24 ft. fga,20-24 ft. fg pct,25-29 ft. fgm,25-29 ft. fga,25-29 ft. fg pct,contested_shots_2pt,contested_shots_3pt,off_boxouts,def_boxouts
0,Atlanta Hawks,2024-25,East,Forward,11.0,16.7,65.7,2.6,5.9,44.5,...,3.4,9.1,37.2,2.9,8.1,35.5,13.9,8.3,0.8,2.7
1,Boston Celtics,2024-25,East,Forward,10.9,16.1,67.4,2.3,5.1,45.2,...,2.9,7.2,39.7,6.6,18.7,35.5,17.1,10.6,1.0,3.8
2,Brooklyn Nets,2024-25,East,Forward,8.1,13.3,61.0,1.9,5.3,36.6,...,3.7,10.5,35.5,5.0,14.0,35.7,11.0,10.1,0.4,2.6
3,Charlotte Hornets,2024-25,East,Forward,7.6,12.4,61.2,2.0,4.9,40.5,...,2.0,5.7,35.5,3.3,10.4,32.1,11.4,8.1,1.2,2.9
4,Chicago Bulls,2024-25,East,Forward,6.3,10.4,60.0,1.3,2.8,47.0,...,2.4,6.6,37.0,2.9,9.0,32.6,11.5,7.0,0.5,2.6


In [32]:
output.to_csv("..\\..\\dataset\\nba_test_dataset.csv", sep=",", index=False)
output.to_csv("..\\..\\dataset\\nba_test_dataset_excel.csv", sep=";", index=False)